# Assignment

Analyze the provided data and answer the questions to the best of your abilities. 
In particular, we are interested in:

- what the gaps are between guest demand and host supply that the new city manager could plug to increase the number of bookings in Dublin,
- what other data would be useful to have to deepen the analysis and understanding.

My Approach:

1. Clean & pre-process with python
2. Use tableau for further visualization

In [ ]:
#necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt
import os
%matplotlib inline
import seaborn as sns
print(pd.__version__)

In [ ]:
# These codes ensure the dataframe is shown fully
import io
pd.set_option('display.max_columns', None)
pd.options.display.max_columns = None

In [ ]:
#Below code shows all the file in my fiule directory
!tar cvfz zipname.tar.gz *

In [ ]:
searches = pd.read_excel("Searches.xlsx")
# iterating the columns
for col in searches.columns:
    print(col)

In [ ]:
country = pd.read_csv("country_csv.csv")

## Merging the full country name with searches dataset 

In [ ]:
country.columns

In [ ]:
#Renaming the column for merging
country.rename(columns={'Code':'origin_country'},inplace = True)

In [ ]:
country.columns

In [ ]:
searches_cn = pd.merge(searches, country, on='origin_country',
                      how='left') 
#new dataset searches with searches_cn title

In [ ]:
#Now Renaming the column for better identificATION 
searches_cn.rename(columns={'Name':'guest_country'},inplace = True)

In [ ]:
# iterating the columns
for col in searches_cn.columns:
    print(col)

### Dropping the code name of countries 

In [ ]:
searches_cn.info()

In [ ]:
# Remove columns as index base
#searches_cn.drop(searches_cn.columns[[8]], axis=1, inplace=True)

In [ ]:
searches_cn.head()

# Handling Missing Values in 'Searches' Dataframe

I will create two seperate dataframes one with all the non-null values of filter_neighborhoods and another without the whole filter_neighborhoods column. 

Because filter_neighborhoods has too many null values working with this will effect other calculation. Again I would like to get some insights from the few filter_neighborhoods values that exists in the dataset. Thus creating two sperate df for clarity.

# Neighborhood Only dataframe

In [ ]:
neighborhood_only = searches_cn.dropna(subset=['filter_neighborhoods'])

In [ ]:
neighborhood_only['filter_room_types'] = neighborhood_only['filter_room_types'].str.strip(',')

In [ ]:
neighborhood_only['filter_neighborhoods'] = neighborhood_only['filter_neighborhoods'].str.strip(',')

In [ ]:
neighborhood_only = neighborhood_only.dropna(subset=['ds_checkin'])

In [ ]:
neighborhood_only.head()

In [ ]:
neighborhood_only.info()

In [ ]:
neighborhood_only.isna().sum() 

I will fill the null values in filter_price_min & filter_price_max with median values. Because there are very extreme values in these columns using mean will affect any kind of price calculations

In [ ]:
neighborhood_only['filter_price_max'] = neighborhood_only['filter_price_max'].fillna((neighborhood_only['filter_price_max'].median()))

In [ ]:
neighborhood_only['filter_price_min'] = neighborhood_only['filter_price_min'].fillna((neighborhood_only['filter_price_min'].median()))

In [ ]:
neighborhood_only.isna().sum() 

Lastly I will fill the filter_room_types with blank space rather than dropping the na's

In [ ]:
neighborhood_only[['filter_room_types']]  = neighborhood_only[['filter_room_types']] .fillna('')


In [ ]:
neighborhood_only = neighborhood_only.dropna(subset=['guest_country'])

In [ ]:
neighborhood_only.info()

In [ ]:
neighborhood_only.isna().sum() 

### Pandas Profiling for neighborhood_only dataset

In [ ]:
pip install pandas-profiling

In [ ]:
from pandas_profiling import ProfileReport
prof = ProfileReport(neighborhood_only)
prof.to_file(output_file='output.html')

In [ ]:
prof

# Finding From Exploratory Analysis for neighborhood_only df

- Average #nights 6.59
- Most popular home type 'Entire home/apt'
- Most popular neighborhood City Centre
- Top country for searching Dublin Airbnb is USA
- There is a correlation between 'filter_room_types' & 'n_searches'

# Visualization in mitosheets neighborhood_only

In [ ]:
pip install dash

In [ ]:
import mitosheet
mitosheet.sheet(analysis_to_replay="id-gihiogiwvl")

# Searches Dataframe

In [ ]:
searches_cn = searches_cn.drop('filter_neighborhoods', axis=1)

In [ ]:
#searches_cn = searches_cn.drop('filter_room_types', axis=1)

In [ ]:
searches_cn['filter_room_types'] = searches_cn['filter_room_types'].str.strip(',')

In [ ]:
searches_cn['filter_price_max'] = searches_cn['filter_price_max'].fillna((searches_cn['filter_price_max'].median()))


In [ ]:
searches_cn['filter_price_min'] = searches_cn['filter_price_min'].fillna((searches_cn['filter_price_min'].median()))


In [ ]:
searches_cn.info()

In [ ]:
searches_cn.head()

In [ ]:
searches_cn = searches_cn.dropna(subset=['ds_checkin'])

In [ ]:
searches_cn = searches_cn.dropna(subset=['guest_country'])

In [ ]:
searches_cn[['filter_room_types']]  = searches_cn[['filter_room_types']] .fillna('')

In [ ]:
searches_cn.isna().sum() 

In [ ]:
searches_cn.info()

### Pandas Profiling for searches_cn dataset

In [ ]:
from pandas_profiling import ProfileReport
prof_2 = ProfileReport(searches_cn)
prof_2.to_file(output_file='output2.html')

In [ ]:
prof_2

In [ ]:
import mitosheet
mitosheet.sheet(analysis_to_replay="id-zguulnwewg")

### Pandas Profiling for searches_cn dataset

In [ ]:
from pandas_profiling import ProfileReport
prof_2 = ProfileReport(searches_cn)
prof_2.to_file(output_file='output2.html')

In [ ]:
prof_2

# Contacts Dataset

In [ ]:
contacts = pd.read_excel("Contacts.xlsx")

In [ ]:
contacts

In [ ]:
contacts.isna().sum()

In [ ]:
contacts = contacts.dropna(subset=['ts_reply_at'])

In [ ]:
contacts.isna().sum()

In [ ]:
contacts['tod_guest'] = pd.cut(pd.to_datetime(contacts.ts_contact_at).dt.hour,
       bins=[0, 6, 12, 18, 24],
       labels=['night', 'morning', 'afternoon', 'evening'],
       right=False,
       include_lowest=True)


In [ ]:
contacts['length_of_stay'] = contacts['ds_checkout'] - contacts['ds_checkin']

In [ ]:
#converting timedelta64 to days
contacts['length_of_stay'] = contacts['length_of_stay'].astype('timedelta64[D]')


In [ ]:
# add a timedelta column if wanted. It's added here for information only
# df['time_delta_with_sub'] = df.from_date.sub(df.to_date)  # also works
#df['time_delta'] = (df.from_date - df.to_date)
contacts['hours_taken_to_reply_guests'] = contacts['ts_reply_at'] - contacts['ts_contact_at']


In [ ]:
#converting timedelta64 to minutes
contacts['hours_taken_to_reply_guests'] = contacts['hours_taken_to_reply_guests'].astype('timedelta64[h]')


In [ ]:
contacts['accepted'] = np.where(np.isnan(contacts['ts_accepted_at']), False, True)


In [ ]:
contacts.isna().sum()

In [ ]:
import mitosheet
mitosheet.sheet(analysis_to_replay="id-hbaqdogjsp")

### Pandas Profiling for contacts dataset

In [ ]:
from pandas_profiling import ProfileReport
prof_3 = ProfileReport(contacts)
prof_3.to_file(output_file='output3.html')

In [ ]:
prof_3

### Download Excel File

In [ ]:
pip install openpyxl

In [ ]:
neighborhood_only.to_excel("neighborhood_only.xlsx", encoding="utf-8")

In [ ]:
searches_cn.to_excel("searches_cn.xlsx", encoding="utf-8")

In [ ]:
contacts.to_excel("contacts.xlsx", encoding="utf-8")

Resources Used
- https://towardsdatascience.com/exploratory-data-analysis-with-pandas-profiling-de3aae2ddff3